In [2]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import glob
from functions import convert_wind

def make_1s_file(data):
    #variable dictionary
   
    # variables
    avg_temp = np.round(data['T'].resample('1S', label='right').mean(),1)
    avg_P =    np.round(data['P'].resample('1S', label='right').mean(),1)
    avg_RH =   np.round(data['RH'].resample('1S', label='right').mean(),1)
    avg_ws =   np.round(data['windsp'].resample('1S', label='right').mean(),1)
    times = avg_P.index
    
    # wind direction
    u,v = convert_wind(data['windsp'], data['winddir'])
    u_avg = u.resample('1S', label='right').mean()
    v_avg = v.resample('1S', label='right').mean()
    avg_wd = np.round(np.rad2deg(np.arctan2(u_avg, v_avg))+180,1) # rotate so 0 is N
    
    
    # if 10Hz data used in a minute has been flagged, the whole minute will be flagged
    # if data looks wonky, checking this number may explain why
    tflag = data['tflag'].resample('1S', label='right').max()
    wflag = data['wflag'].resample('1S', label='right').max()
    

    # put data into pandas dataframe
    d = {'T':avg_temp, 'RH':avg_RH, 'P':avg_P, 'WS':avg_ws, 
         'WD':avg_wd, 'TFLAG':tflag, 'WFLAG':wflag}
    avgmet = pd.DataFrame(data=d,index=times).replace(-999.9,np.nan)
    avgmet.sort_index(ascending = True, inplace = True)
    
    return avgmet

In [4]:
# for stesonet
datadir = r'/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/data/'
savedir = r'/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/data_1s/'

col_names = ['time', 'T', 'RH', 'P', 'windsp', 'winddir', 'batt', 'tflag','wflag', 'ID']
allfiles = glob.glob('{}*.csv'.format(datadir))

for f in allfiles:
    data10 = pd.read_csv(f, index_col=0, header=0,names=col_names, 
                         parse_dates=['time'], date_parser=pd.to_datetime)
    if data10.empty != True:
        data1 = make_1s_file(data10)
        fname = f[-18:]
        data1.to_csv('{}{}'.format(savedir, fname))

In [3]:
# modify this for single sticknets

datadir = r'/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/data/'
savedir = r'/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/data_1s/'

col_names = ['time', 'T', 'RH', 'P', 'windsp', 'winddir', 'batt', 'tflag','wflag', 'ID']
allfiles = glob.glob('{}0217A_20170301.csv'.format(datadir))

for f in allfiles:
    data10 = pd.read_csv(f, index_col=0, header=0,names=col_names, 
                         parse_dates=['time'], date_parser=pd.to_datetime)
    if data10.empty != True:
        data1 = make_1s_file(data10)
        fname = f[-18:]
        data1.to_csv('{}{}'.format(savedir, fname))

In [100]:
# for rapid probes

class SNFile(object):
    def __init__(self, filename):
        self.year = int(filename[-19:-15])
        self.month = int(filename[-15:-13])
        self.day = int(filename[-13:-11])
        self.hour = int(filename[-10:-8])
        self.minute = int(filename[-8:-6])
        self.datetime = dt.datetime(self.year,self.month,self.day)
        self.filename = filename
        self.probe = filename[-25:-21]
        f = open(self.filename)
        header = f.readline().split()
        self.header = header[0]
        f.close()
      
    def parse(self, time):
        
        try:
            timep = "%08.1f"%time
            if timep == 000000.0:
                self.datetime += dt.timedelta(days=1) 
            hour,minute,second,msec = int(timep[0:2]),int(timep[2:4]),int(timep[4:6]),int(timep[-1])
            return self.datetime + dt.timedelta(hours=hour,minutes=minute,seconds=second,microseconds=msec*100000)
        except:
            if [self.year, self.month, self.day] == [2017, 3, 1]:
                t = dt.datetime.strptime(time,'%d_%H%M%S.%f')
                t = t.replace(year=self.year, month=self.month)
                return t
            else:
                return dt.datetime.strptime(time,'%Y%m%d_%H%M%S.%f')
    

    def read(self):
        # for rapid probes, that don't have the extra data point at the end
        sn = pd.read_csv(self.filename, names=['time','T','RH','P','windsp','winddir','batt','flag','flag2'],
                         dtype={'time':np.float},parse_dates=[0],date_parser=self.parse,header=1,
                         error_bad_lines = False)
        
        sn['ID'] = pd.Series(self.probe,index=sn.index)
        sn.index = sn['time']
        return sn

In [101]:
basedir = '/Users/jessmcd/Documents/MyPassport_backup/VSE_Data/'
savedir = r'/Users/jessmcd/Documents/MyPassport_backup/VortexSE/NewAnalyses/data_1s/'

rpfiles = []

deployments = ['20160331_deployment', '20160430_deployment',
               '20170301_deployment', '20170325_deployment', '20170327_deployment1', '20170430_deployment']
for dep in deployments:
    if dep[0:4] == '2016':
        rpfiles.extend(glob.glob('{}2016/Deployments/{}/reformat/*.txt'.format(basedir, dep)))
    else: # 2017
        rpfiles.extend(glob.glob('{}2017/deployments/{}/reformat/*.txt'.format(basedir, dep)))
        
for f in rpfiles:
    data_ob = SNFile(f)
    data10 = data_ob.read()
    data10.rename(columns={'flag':'tflag', 'flag2':'wflag'}, inplace=True)
    if data10.empty != True:
        data1 = make_1s_file(data10)
        fname = f[-25:-11]
        data1.to_csv('{}{}.csv'.format(savedir, fname))

In [ ]:
## dealing with 2017 March 1 217A issue ... data file was never QC'd....